In [1]:
!pip install colorednoise

In [2]:
from google.colab import drive
import zipfile
import os
import pickle
import numpy as np
from tqdm import tqdm  #for the progress bar
import torch
import shutil
from pathlib import Path
import re
import matplotlib.pyplot as plt
import colorednoise as cn


Connecting to Google Drive and Specifying Paths

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Unzip the uploaded folder
#zip_path = '/content/drive/MyDrive/HRTF_noise/hr.zip'
unzip_path = '/content/drive/MyDrive/HRTF_noise/hrtf_folder/hr'
modified_folder_path = '/content/drive/MyDrive/HRTF_noise/brown_noise/-50dB/modified_hrtf_folder'

if not os.path.exists(modified_folder_path):
    os.makedirs(modified_folder_path)

In [ ]:
def generate_noise(hrir_shape, noise_type):
    """
    Generates multi-dimensional noise based on the input shape and noise type.

    Args:
        hrir_shape (tuple): Shape of the desired noise (num_panels, panel_height, panel_width, num_freqs).
        noise_type (int): Type of noise to generate. 0 for white noise, 1 for pink noise, 2 for brown noise.

    Returns:
        Generated noise tensor with the same shape as `hrir_shape`.
    """

    num_panels, panel_height, panel_width, num_freqs = hrir_shape
    noise = torch.empty(hrir_shape)

    for i in range(num_panels):
        for j in range(panel_height):
            for k in range(panel_width):
                # Generate pink noise for each position separately (in the time domain)
                noise[i, j, k, :] = torch.tensor(cn.powerlaw_psd_gaussian(noise_type, num_freqs))

    return noise

In [ ]:
def add_noise_to_hrtf(hrir, noise, snr_db):
    """Adds noise to the HRTF (in the frequency domain) with a specified signal-to-noise ratio (SNR),
       operating in the time domain.

    Args:
        hrir: The HRIR tensor (assumed to be in the time domain).
        noise: The pink noise tensor (already in the time domain).
        snr_db: The desired signal-to-noise ratio in decibels.

    Returns:
        The modified HRTF tensor with the added noise, back in the frequency domain.
    """
    #for debugging purposes
    #print(f"the shape of hrir is{hrir.shape} and that of noise {noise.shape}")

    # Calculate the RMS amplitude of the HRIR and noise in the time domain
    hrir_rms = torch.sqrt(torch.mean(hrir**2, dim=-1, keepdim=True))
    noise_rms = torch.sqrt(torch.mean(noise**2, dim=-1, keepdim=True))

    # Convert desired SNR from dB to a linear scale (amplitude)
    snr_linear = 10 ** (snr_db / 20)

    # Calculate the scaling factor for the noise
    #for debugging purposes
    #print(f"the shape of hrir_rms is {hrir_rms.shape} and of noise is {noise_rms.shape}")
    noise_scaling_factor = hrir_rms / (noise_rms * snr_linear)

    # Apply the scaling factor to the noise
    noise_scaled = noise * noise_scaling_factor

    #for debugging purposes
    #print(f"the size of the hrir is {hrir.size} and the size of the noise is {noise.size}")

    # Add the scaled noise to the HRIR in the time domain
    modified_hrir = hrir + noise_scaled

    # Convert the modified HRIR back to the frequency domain (HRTF)
    modified_hrtf = torch.abs(torch.fft.rfft(modified_hrir, dim=-1))
    #print(f"the shape of the modified hrtf is {modified_hrtf.shape}")

    #for debugging purposes
    #if (modified_hrtf < 0).any():
    #  print("modified hrtf negative ")

#    assert(modified_hrtf.shape == hrtf.shape), f"Shape different: modified {modified_hrtf.shape} != original {hrtf.shape}"

    return modified_hrtf

In [ ]:
# Function to save the modified HRTF to a pickle file
def save_hrtf(hrtf, file_path):
    with open(file_path, 'wb') as file:
        if(pickle.dump(hrtf, file, protocol=4)):
          print("successful")


In [ ]:
"""
Function taken from the original work non upsampling, which can be found at:
https://github.com/ahogg/HRTF-upsampling-with-a-generative-adversarial-network-using-a-gnomonic-equiangular-projection/tree/main/evaluation
"""
def merge_left_right_hrtfs(input_dir, output_dir):
    # Clear/Create directory
    shutil.rmtree(Path(output_dir), ignore_errors=True)
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    hrtf_file_names = [os.path.join(input_dir, hrtf_file_name) for hrtf_file_name in os.listdir(input_dir)
                       if os.path.isfile(os.path.join(input_dir, hrtf_file_name))]

    data_dict_left = {}
    data_dict_right = {}
    for f in hrtf_file_names:

        file_ext = re.findall(re.escape(input_dir) + '/(.*)_[0-9]*[a-z]*.pickle$', f)[0]

        with open(f, "rb") as file:
            data = pickle.load(file)

        # add to dict for right ears
        if re.search(re.escape(input_dir)+'/.*_[0-9]*right.pickle$', f):
            subj_id = int(re.findall(re.escape(input_dir)+'/.*_([0-9]*)right.pickle$', f)[0])
            if file_ext not in data_dict_right:
                data_dict_right[file_ext] = {}
            data_dict_right[file_ext][subj_id] = data
        # add to dict for left ears
        elif re.search(re.escape(input_dir)+'/.*_[0-9]*left.pickle$', f):
            subj_id = int(re.findall(re.escape(input_dir)+'/.*_([0-9]*)left.pickle$', f)[0])
            if file_ext not in data_dict_left:
                data_dict_left[file_ext] = {}
            data_dict_left[file_ext][subj_id] = data

    for file_ext in data_dict_right.keys():
        missing_subj_ids =  list(set(data_dict_right[file_ext].keys()) - set(data_dict_left[file_ext].keys()))
        if len(missing_subj_ids) > 0:
            print('Excluding subject IDs where both ears do not exist (IDs: %s)' % ', '.join(map(str, missing_subj_ids)))
            for missing_subj_id in missing_subj_ids:
                data_dict_right[file_ext].pop(missing_subj_id, None)
                data_dict_left[file_ext].pop(missing_subj_id, None)

        for subj_id in data_dict_right[file_ext].keys():
            hrtf_r = data_dict_right[file_ext][subj_id]
            hrtf_l = data_dict_left[file_ext][subj_id]
            dimension = hrtf_r.ndim-1
            hrtf_merged = torch.cat((hrtf_l, hrtf_r), dim=dimension)
            with open('%s/%s_%s.pickle' % (output_dir, file_ext, subj_id), "wb") as file:
                pickle.dump(hrtf_merged, file,protocol=4)

In [ ]:
"""
Main loop that goes through every HRTF in the train and valid folders
"""

for subfolder in ["train", "valid"]:
    subfolder_path = os.path.join(unzip_path, subfolder)
    modified_subfolder_path = os.path.join(modified_folder_path, subfolder)
    merged_subfolder_path = os.path.join(modified_folder_path, f"merged_{subfolder}")

    # Create directories if they don't exist
    os.makedirs(modified_subfolder_path, exist_ok=True)
    os.makedirs(merged_subfolder_path, exist_ok=True)
    for filename in tqdm(os.listdir(subfolder_path), desc=f"Processing {subfolder}"):

        if filename.endswith(".pickle"):
            file_path = os.path.join(subfolder_path, filename)
            new_file_path = os.path.join(modified_subfolder_path, filename)
            try:
                with open(file_path, 'rb') as file:  # Use context manager to open file
                    hrtf = pickle.load(file)

                # convert the HRTF to HRIR
                hrir = torch.fft.irfft(hrtf, dim=-1)

                noise = generate_noise(hrir.shape, 2)

                modified_hrtf = add_noise_to_hrtf(hrir, noise, snr_db=-50)

                # Save the modified HRTF
                #save_hrtf(modified_hrtf, new_file_path)
            except (pickle.UnpicklingError, EOFError) as e:  # Catch errors during load
                print(f"Error loading pickle file '{filename}': {e}")
            except Exception as e:  # Catch other errors
                print(f"Error processing file '{filename}': {e}")
     #Merge left and right HRTFs after processing all files in the subfolder
    merge_left_right_hrtfs(modified_subfolder_path, merged_subfolder_path)


Processing train:   4%|▍         | 13/324 [00:00<00:04, 63.59it/s]

torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_153right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_12left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_43right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_140left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_146left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_180right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_65right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_110left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Son

Processing train:  10%|▉         | 32/324 [00:00<00:03, 81.69it/s]

torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_17right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_194right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_138right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_184right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_28right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_13left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_120left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_56left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Son

Processing train:  16%|█▌        | 52/324 [00:00<00:03, 89.74it/s]

...

Processing train:  52%|█████▏    | 169/324 [00:01<00:01, 89.77it/s]

torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_119left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_61right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_133left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_77right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_126left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_38left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_135left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_107left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonic

Processing train:  53%|█████▎    | 173/324 [00:01<00:01, 86.66it/s]

torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_121right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_1right.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_52left.pickle': name 'generate_noise' is not defined
torch.Size([5, 16, 16, 254])
Error processing file 'Sonicom_mag_156left.pickle': name 'generate_noise' is not defined


KeyboardInterrupt: 

In [ ]:
"""
Code used to understand problems during the loading of pickle files. The error
encountered was that whenever a pickle file would load, there would be an invalid load key
error because the files were opened on macOS, which created an invisible .DS_STORE file.
"""
import os
import pickle

def load_hrtf(file_path):
    with open(file_path, 'rb') as file:
        hrtf = pickle.load(file)
    return hrtf

def verify_pickle_files(directory):
    count = 0
    for filename in os.listdir(directory):
        if filename.endswith('.pickle'):
            file_path = os.path.join(directory, filename)
            try:
                hrtf = load_hrtf(file_path)
                count+=1
                print(count)
                print(f"Successfully loaded {file_path}")
            except pickle.UnpicklingError as e:
                print(f"Failed to load {file_path}: {e}")
            except Exception as e:
                print(f"An unexpected error occurred while loading {file_path}: {e}")

# Change this to the directory containing your pickle files
pickle_directory = '/content/drive/MyDrive/HRTF_noise/pink_noise/0dB/modified_hrtf_folder/merged_train'
verify_pickle_files(pickle_directory)

1
Successfully loaded /content/drive/MyDrive/HRTF_noise/pink_noise/0dB/modified_hrtf_folder/merged_train/Sonicom_mag_153.pickle
2
Successfully loaded /content/drive/MyDrive/HRTF_noise/pink_noise/0dB/modified_hrtf_folder/merged_train/Sonicom_mag_43.pickle
3
Successfully loaded /content/drive/MyDrive/HRTF_noise/pink_noise/0dB/modified_hrtf_folder/merged_train/Sonicom_mag_180.pickle
4
Successfully loaded /content/drive/MyDrive/HRTF_noise/pink_noise/0dB/modified_hrtf_folder/merged_train/Sonicom_mag_65.pickle
5
Successfully loaded /content/drive/MyDrive/HRTF_noise/pink_noise/0dB/modified_hrtf_folder/merged_train/Sonicom_mag_94.pickle
6
Successfully loaded /content/drive/MyDrive/HRTF_noise/pink_noise/0dB/modified_hrtf_folder/merged_train/Sonicom_mag_139.pickle
7
Successfully loaded /content/drive/MyDrive/HRTF_noise/pink_noise/0dB/modified_hrtf_folder/merged_train/Sonicom_mag_13.pickle
8
Successfully loaded /content/drive/MyDrive/HRTF_noise/pink_noise/0dB/modified_hrtf_folder/merged_train/Son